In [1]:
from pyscipopt import Model, quicksum, multidict
import pandas as pd
import numpy as np

In [2]:
# Loading data source

# 1. WW build qty
df_build = pd.read_csv('WW_buildplan.tab', sep='\t')
df_build = df_build[df_build['week']=='WW05']
bp = dict(zip(df_build['prod'], df_build['build_qty']))


# 2. Inventory

df_inventory = pd.read_csv('inventory_filtered.tab', sep='\t')


# 3. Config file: vendor qualification & part qty per unit

df_vendor_qual = pd.read_excel('config.xlsx', 'vendor_qualification')
df_part_qty = pd.read_excel('config.xlsx', 'part_per_unit')
df_part_qty['demand'] = df_part_qty.apply(lambda x: x['part_qty_per_unit'] * (bp[x['prodgroup3']]), axis=1)

In [3]:
def get_demand(prod, part_num):
    try:
        demand = df_part_qty.loc[(df_part_qty['prodgroup3'] == prod) & (df_part_qty['part_number'] == part_num), 'demand'].values[0]
    except:
        demand = 0
    return demand

In [4]:
def get_inventory(vendor, part_num):
    
    try:
        inv = df_inventory[(df_inventory['itemid']==part_num)&(df_inventory['vendorname'].str.contains(vendor).fillna(False))]['inv_qty'].values[0]
    except:
        inv = 0
        
    return inv

In [5]:
# Solver


model = Model('single_vendor_mapping')


# define a big-M: 
M = df_part_qty.demand.sum()


# variables
# x_ijk --> i: prod; j: part_number; k: vendor (Continuous)
# y_ijk --> i: prod; j: part_number; k: vendor (Boolean)


x, y = {}, {}

for idx, row in df_vendor_qual.iterrows():
    i, j, k = row['prodgroup3'], row['part_number'], row['qual_vendor']
    x[i, j, k] = model.addVar(vtype="C", name="x(%s,%s,%s)" % (i, j, k))
    y[i, j, k] = model.addVar(vtype="B",name="y(%s,%s,%s)" % (i, j, k))


# Cons1: sum(x_ijk/per part_number&vendor) <= inventory

for j in set(df_vendor_qual['part_number']):
    for k in set(df_vendor_qual[df_vendor_qual['part_number']==j]['qual_vendor']):
        inv = get_inventory(k, j)
        prods = set(df_vendor_qual[(df_vendor_qual['part_number']==j)&(df_vendor_qual['qual_vendor']==k)]['prodgroup3'])
        model.addCons(quicksum(x[i, j, k] for i in prods) <= inv)

        
# Cons2: sum(x_ijk per part_number&prod) == prod demand for that part

for i in set(df_vendor_qual['prodgroup3']):
    for j in set(df_vendor_qual[df_vendor_qual['prodgroup3']==i]['part_number']):
        demand = get_demand(i, j)
        vendors = set(df_vendor_qual[(df_vendor_qual['prodgroup3']==i)&(df_vendor_qual['part_number']==j)]['qual_vendor'])
        model.addCons(quicksum(x[i, j, k] for k in vendors) == demand)
        
        
# Cons3: single vendor -> for each prod for each part: only one vendor can be selected

for i in set(df_vendor_qual['prodgroup3']):
    for j in set(df_vendor_qual[df_vendor_qual['prodgroup3']==i]['part_number']):
        vendors = set(df_vendor_qual[(df_vendor_qual['prodgroup3']==i)&(df_vendor_qual['part_number']==j)]['qual_vendor'])
        model.addCons(quicksum(y[i, j, k] for k in vendors) == 1)


# Cons4: only selected vendor can support the demand

for i in set(df_vendor_qual['prodgroup3']):
    for j in set(df_vendor_qual[df_vendor_qual['prodgroup3']==i]['part_number']):
        vendors = set(df_vendor_qual[(df_vendor_qual['prodgroup3']==i)&(df_vendor_qual['part_number']==j)]['qual_vendor'])
        for k in vendors:
            model.addCons((x[i, j, k]/M) <= y[i, j, k])

model.optimize()

print(model.getStatus())

optimal


In [6]:
# Save result

df_result = pd.DataFrame(columns=['prod', 'part_num', 'vendor', 'selected', 'qty'])

for idx, row in df_vendor_qual.iterrows():
    i, j, k = row['prodgroup3'], row['part_number'], row['qual_vendor']
    
    df_result.loc[len(df_result.index)] = [i, j, k, model.getVal(y[(i, j, k)]), model.getVal(x[(i, j, k)])] 
    
df_result['selected'] = df_result['selected'].replace(1, 'Y')
df_result['selected'] = df_result['selected'].replace(0, '')
df_result['qty'] = df_result['qty'].replace(0, '')

for idx, row in df_result.iterrows():
    if row['selected'] != '':
        vendor = row['vendor']
        vendor_code = df_vendor_qual[df_vendor_qual['qual_vendor']==vendor]['vendor_code'].values[0]
        df_result.at[idx, 'Lot Attribute'] = row['part_num'] + str(vendor_code)
    else:
        df_result.at[idx, 'Lot Attribute'] = ''

df_result = df_result.sort_values(by=['prod', 'part_num'])

In [10]:
df_result
# df_result[df_result['selected']=='Y']

,prod,part_num,vendor,selected,qty,Lot Attribute
1,PKR,C48956-004,MURATA,Y,15000.0,C48956-00461
2,PKR,C48956-010,MURATA,Y,15000.0,C48956-01061
0,PKR,J66920-002,MURATA,Y,15000.0,J66920-00261
4,SKR,C48956-004,MURATA,Y,10000.0,C48956-00461
5,SKR,C48956-004,AVX,,,
6,SKR,C48956-010,MURATA,Y,10000.0,C48956-01061
7,SKR,C48956-010,SEMCO,,,
3,SKR,J66920-002,MURATA,Y,10000.0,J66920-00261


In [ ]:
'''
-----------------------------------------------------------------
 BATCH |     LA7100       |     LA7101       |     LA7102       |
----------------------------------------------------------------|
  PKR  |   C48956-00461   |   C48956-01061   |   J66920-00261   |
----------------------------------------------------------------|
  SKR  |   C48956-00461   |   C48956-01061   |   J66920-00261   |
-----------------------------------------------------------------
'''